In [9]:
import cv2
import numpy as np

In [ ]:
def binary_image(img_path: str):
    """This function would return the inverted binary image"""

    img = cv2.imread(img_path)

    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_blur = cv2.GaussianBlur(img_gray, (7, 7), 0)

    thresh = cv2.adaptiveThreshold(
        img_blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2
    )

    return thresh


def find_contours(binary_img: str):

    result = cv2.findContours(binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = result[0] if len(result) == 2 else result[1]
    contours = sorted(contours, key=cv2.contourArea, reverse=True)

    return contours


bimg = binary_image("../imgs/620sodukujan2013.jpg")
cnts = find_contours(binary_img=bimg)

print(cnts)


def approx_to_quadrants(cnt: np.array):
    """Try to approximate a contour to 4 points; return quad or None."""
    perimeter = cv2.arcLength(cnt, True)

    for fraction in (0.02, 0.03, 0.04, 0.05, 0.06):
        approx = cv2.approxPolyDP(cnt, fraction * perimeter, True)

        if len(approx) == 4 and cv2.isContourConvex(approx):
            return approx.astype(np.float32)

    return None


def pick_largest_quadrant(contours, min_area_ratio=0.10, image_area=None):
    best_quad, best_area = None, 0.0

    for cnt in contours:
        area = cv2.contourArea(cnt)

        if image_area is not None and area < min_area_ratio * image_area:
            continue

        quad = approx_to_quadrants(cnt)
        if quad is None:
            continue
        if area > best_area:
            best_area, best_quad = area, quad

    return best_quad


def reorder_points(points: np.array):
    pts = points.reshape((4, 2))
    s = pts.sum(axis=1)
    d = np.diff(pts, axis=1)

    ordered = np.zeros((4, 2), dtype=np.float32)
    ordered[0] = pts[np.argmin(s)]  # TL
    ordered[2] = pts[np.argmax(s)]  # BR
    ordered[1] = pts[np.argmin(d)]  # TR
    ordered[3] = pts[np.argmax(d)]  # BL

    return ordered


def draw_quad(img, quad, color=(0, 255, 0), thickness=3):
    out = img.copy()
    q = quad.astype(int).reshape(4, 2)
    # draw edges
    for i in range(4):
        p1 = tuple(q[i])
        p2 = tuple(q[(i + 1) % 4])
        cv2.line(out, p1, p2, color, thickness)
    # draw corners
    for p in q:
        cv2.circle(out, tuple(p), 6, (0, 0, 255), -1)
    return out

[array([[[  0,   0]],

       [[  0,   1]],

       [[ 61,   1]],

       ...,

       [[140,   1]],

       [[199,   1]],

       [[199,   0]]], dtype=int32), array([[[240,  17]],

       [[239,  18]],

       [[236,  18]],

       [[235,  19]],

       [[234,  19]],

       [[233,  20]],

       [[232,  20]],

       [[229,  23]],

       [[229,  24]],

       [[228,  25]],

       [[228,  26]],

       [[227,  27]],

       [[227,  30]],

       [[226,  31]],

       [[226,  40]],

       [[227,  41]],

       [[227,  44]],

       [[228,  45]],

       [[228,  46]],

       [[231,  49]],

       [[232,  49]],

       [[233,  50]],

       [[234,  50]],

       [[235,  51]],

       [[241,  51]],

       [[242,  50]],

       [[244,  50]],

       [[245,  49]],

       [[246,  49]],

       [[249,  46]],

       [[249,  45]],

       [[250,  44]],

       [[250,  35]],

       [[249,  34]],

       [[249,  33]],

       [[246,  30]],

       [[243,  30]],

       [[242,  29]],

    

In [24]:
img = cv2.imread("../imgs/620sodukujan2013.jpg")
bimg = binary_image("../imgs/620sodukujan2013.jpg")

cnts = find_contours(bimg)
quad = pick_largest_quadrant(cnts, image_area=bimg.shape[0] * bimg.shape[1])

if quad is None:
    raise ValueError("No 4-point Sudoku contour found.")

quad_ordered = reorder_points(quad)

debug = draw_quad(img, quad_ordered)

cv2.imshow("Sudoku Outline", debug)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [32]:
import easyocr

reader = easyocr.Reader(["en"])
img_path = "../imgs/img.png"

original_img = cv2.imread(img_path)

grayscale_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2GRAY)

blur_img = cv2.GaussianBlur(grayscale_img, (7, 7), 0)
thresh_img = cv2.adaptiveThreshold(
    blur_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2
)

result = reader.readtext(thresh_img, detail=0, paragraph=False)
print(result)

digits = []

if len(result) > 0:
    text = result[0]

    if text.isdigit():
        digits.append(int(text))

for digit in digits:
    print(digit)

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


['4', '5', '2', '7', '6', '3', '2', '8', '9 | 5', '8', '61', '2', '7 | 5', '4 | 7', '6', '4 | 5', '8', 'Sudoku Puzzle #3', '36']
4


In [2]:
import cv2
import numpy as np


def show_sudoku_contours(img_path):
    """
    Loads an image, finds the Sudoku grid, and draws its contour and corners.
    It also finds and draws the contours of the numbers within the grid.

    Args:
        img_path (str): The path to the image file.
    """
    # Load and preprocess the image
    img = cv2.imread(img_path)
    if img is None:
        print(f"Error: Could not load image at {img_path}")
        return

    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Image thresholding for the main grid
    img_thresh = cv2.adaptiveThreshold(
        img_gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2
    )

    # Find and select the main grid contour
    contours, _ = cv2.findContours(
        img_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )
    if not contours:
        print("Error: No contours found.")
        return

    contour = max(contours, key=cv2.contourArea)
    peri = cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, 0.02 * peri, True)

    # Draw the main grid contour in green and its corners in red
    output_img = img.copy()
    cv2.drawContours(output_img, [approx], -1, (0, 255, 0), 3)

    for point in approx.reshape(-1, 2):
        cv2.circle(output_img, (point[0], point[1]), 10, (0, 0, 255), -1)

    # Isolate the numbers by applying a different threshold
    # Note: A simple binary threshold often works better for isolating numbers
    ret, number_thresh = cv2.threshold(img_gray, 150, 255, cv2.THRESH_BINARY_INV)

    # Find contours for the numbers
    number_contours, _ = cv2.findContours(
        number_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )

    # Filter and draw the number contours in yellow
    for c in number_contours:
        # Filter out small contours that are likely noise
        area = cv2.contourArea(c)
        if area > 100:  # Adjust this value based on image resolution
            # Filter out the main grid contour to avoid drawing over it
            x, y, w, h = cv2.boundingRect(c)
            # A simple heuristic to check if the contour's dimensions are reasonable for a number
            if w > 10 and h > 10 and w < 100 and h < 100:
                cv2.drawContours(output_img, [c], -1, (0, 255, 255), 2)  # Yellow color

    # Display the final result
    cv2.imshow("Sudoku Contours with Numbers", output_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


# Example usage
# Ensure 'path_to_your_image.jpg' points to a valid image file.
show_sudoku_contours("../imgs/img.png")